# Neural Reranking Implementation for BioASQ

In [ ]:
# This builds on the existing neural approach, adding a reranking stage

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import json
import tqdm

from utils import load_bioasq_test_questions
from utils_neural import retrieve_and_rank_documents_neural, extract_and_rank_snippets_neural

In [ ]:
# Additional function for reranking
def load_reranker_model():
    """Load cross-encoder reranker model (BioBERT-based)"""
    MODEL_NAME = "pritamdeka/BioBERT-Reranker"  # Or another biomedical cross-encoder model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    return model, tokenizer, device


In [ ]:
def rerank_documents(question, candidate_docs, model, tokenizer, device, batch_size=8):
    """
    Reranks candidate documents using a cross-encoder model
    
    Args:
        question (dict): Question dictionary
        candidate_docs (list): List of document dictionaries from first-stage retrieval
        model: Cross-encoder reranker model
        tokenizer: Tokenizer for the model
        device: Computation device
        
    Returns:
        list: Reranked list of document dictionaries
    """
    if not candidate_docs:
        return []
    
    documents = candidate_docs
    query = question['body']
    
    # Prepare text pairs for the cross-encoder
    text_pairs = []
    for doc in documents:
        # Combine title and abstract for the document text
        doc_text = f"{doc['title']} {doc['documentAbstract']}"
        text_pairs.append((query, doc_text))
    
    # Get relevance scores in batches
    all_scores = []
    
    for i in range(0, len(text_pairs), batch_size):
        batch_pairs = text_pairs[i:i+batch_size]
        
        # Tokenize and format inputs for the cross-encoder
        inputs = tokenizer(
            [pair[0] for pair in batch_pairs],
            [pair[1] for pair in batch_pairs],
            padding=True,
            truncation='longest_first',
            max_length=512,
            return_tensors='pt'
        ).to(device)
        
        # Get relevance scores
        with torch.no_grad():
            outputs = model(**inputs)
            # For binary classification models, take the probability of the positive class
            if outputs.logits.shape[1] == 2:
                batch_scores = torch.softmax(outputs.logits, dim=1)[:, 1].cpu().numpy()
            else:
                # For regression models or other formats
                batch_scores = outputs.logits.flatten().cpu().numpy()
        
        all_scores.extend(batch_scores)
    
    # Combine documents with their cross-encoder scores
    reranked_docs = [(doc, score) for doc, score in zip(documents, all_scores)]
    
    # Sort by decreasing score
    reranked_docs.sort(key=lambda x: x[1], reverse=True)
    
    return reranked_docs


In [ ]:
def process_bioasq_questions(test_questions_file, output_file, first_stage_candidates=50, final_docs=10, max_snippets=10):
    """
    Process BioASQ test questions and generate answers using the multi-stage pipeline.
    
    Args:
        test_questions_file (str): Path to test questions file
        output_file (str): Path to output file for results
        first_stage_candidates (int): Number of candidates to retrieve in first stage
        final_docs (int): Number of documents to return per question
        max_snippets (int): Maximum number of snippets to return per question
    """
    # Load test questions
    test_questions = load_bioasq_test_questions(test_questions_file)
    print(f"Loaded {len(test_questions)} test questions")
    
    # Load models - reuse the same BioBERT model for first-stage retrieval
    print("Loading BioBERT model for first-stage retrieval...")
    MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
    retrieval_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    retrieval_model = AutoModel.from_pretrained(MODEL_NAME)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    retrieval_model = retrieval_model.to(device)
    
    print("Loading reranker model...")
    reranker_model, reranker_tokenizer, _ = load_reranker_model()
    
    # Process questions
    results = []
    
    for question in tqdm.tqdm(test_questions, desc="Processing questions"):
        print(f"\nProcessing question: {question['id']} - {question['body']}")
        
        # Stage 1: First-stage retrieval (using your existing function)
        print("  Retrieving candidate documents...")
        # Your existing function returns documents with similarity scores
        ranked_docs = retrieve_and_rank_documents_neural(
            question, 
            retrieval_model, 
            retrieval_tokenizer, 
            max_docs=first_stage_candidates
        )
        
        # Extract just the documents (without scores) for reranking
        candidate_docs = ranked_docs[:first_stage_candidates]
        
        if not candidate_docs:
            print(f"  Warning: No documents found for question {question['id']}")
            # Create empty result
            question_result = {
                'id': question['id'],
                'documents': [],
                'snippets': []
            }
            results.append(question_result)
            continue
        
        print(f"  Retrieved {len(candidate_docs)} candidate documents")
        
        # Stage 2: Rerank documents using cross-encoder
        print("  Reranking documents...")
        reranked_docs = rerank_documents(
            question,
            candidate_docs,
            reranker_model,
            reranker_tokenizer,
            device
        )
        
        # Select top documents
        top_docs = [doc for doc, _ in reranked_docs[:final_docs]]
        
        # Stage 3: Extract and rank snippets (using your existing function)
        print("  Extracting and ranking snippets...")
        ranked_snippets = extract_and_rank_snippets_neural(
            question,
            top_docs,
            retrieval_model,
            retrieval_tokenizer,
            max_snippets=max_snippets
        )
        
        # Format results
        question_result = {
            'id': question['id'],
            'documents': [f"http://www.ncbi.nlm.nih.gov/pubmed/{doc['pmid']}" for doc in top_docs],
            'snippets': ranked_snippets
        }
        
        results.append(question_result)
        
        print(f"  Found {len(top_docs)} documents and {len(ranked_snippets)} snippets")
    
    # Save results
    with open(output_file, 'w') as f:
        json.dump({'questions': results}, f, indent=2)
    
    print(f"Results saved to {output_file}")

In [ ]:
# Run the pipeline
process_bioasq_questions(
    "../data/BioASQ-task13bPhaseA-testset4.txt",
    "BioASQ-task13b-phaseA-testset4-reranker-results.json",
    first_stage_candidates=50,
    final_docs=10,
    max_snippets=10
)